**NOVAJI INTROSERVE ASSESSMENT**

In [1]:
pip install cryptography

Note: you may need to restart the kernel to use updated packages.


In [5]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
from base64 import b64encode, b64decode
import os

# Key and IV generation (AES-256 requires a 32-byte key)
key = os.urandom(32)  # 256-bit key
iv = os.urandom(16)   # AES block size is 16 bytes

# Text to be encrypted
text = 'Welcome to Lagos'.encode('utf-8')

# Padding the text to match AES block size
padder = padding.PKCS7(128).padder()  # AES block size is 128 bits
padded_text = padder.update(text) + padder.finalize()

# AES Encryption
cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update(padded_text) + encryptor.finalize()

# Convert ciphertext to HEX and Base64
ciphertext_hex = ciphertext.hex()
ciphertext_base64 = b64encode(ciphertext).decode('utf-8')

print(f"Ciphertext in HEX: {ciphertext_hex}")
print(f"Ciphertext in Base64: {ciphertext_base64}")

# Decryption (from HEX)
cipher_from_hex = bytes.fromhex(ciphertext_hex)
decryptor = cipher.decryptor()
decrypted_padded_text = decryptor.update(cipher_from_hex) + decryptor.finalize()

# Unpadding after decryption (for HEX)
unpadder = padding.PKCS7(128).unpadder()  # Create a new unpadder instance
decrypted_text = unpadder.update(decrypted_padded_text) + unpadder.finalize()

print(f"Decrypted text from HEX: {decrypted_text.decode('utf-8')}")

# Decryption (from Base64)
cipher_from_base64 = b64decode(ciphertext_base64)
decryptor = cipher.decryptor()
decrypted_padded_text = decryptor.update(cipher_from_base64) + decryptor.finalize()

# Unpadding after decryption (for Base64)
unpadder = padding.PKCS7(128).unpadder()  # Create another unpadder instance
decrypted_text = unpadder.update(decrypted_padded_text) + unpadder.finalize()

print(f"Decrypted text from Base64: {decrypted_text.decode('utf-8')}")


Ciphertext in HEX: 2a174a5d2ce1950c8f9cd754900cdef24e9ee78e8c1df7f1f394cd821e440028
Ciphertext in Base64: KhdKXSzhlQyPnNdUkAze8k6e546MHffx85TNgh5EACg=
Decrypted text from HEX: Welcome to Lagos
Decrypted text from Base64: Welcome to Lagos


In [2]:
pip install django

  Obtaining dependency information for django from https://files.pythonhosted.org/packages/a3/b8/f205f2b8c44c6cdc555c4f56bbe85ceef7f67c0cf1caa8abe078bb7e32bd/Django-5.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for asgiref<4,>=3.8.1 from https://files.pythonhosted.org/packages/39/e3/893e8757be2612e6c266d9bb58ad2e3651524b5b40cf56761e985a28b13e/asgiref-3.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for sqlparse>=0.3.1 from https://files.pythonhosted.org/packages/5d/a5/b2860373aa8de1e626b2bdfdd6df4355f0565b47e51f7d0c54fe70faf8fe/sqlparse-0.5.1-py3-none-any.whl.metadata
  Obtaining dependency information for tzdata from https://files.pythonhosted.org/packages/a6/ab/7e5f53c3b9d14972843a647d8d7a853969a58aecc7559cb3267302c94774/tzdata-2024.2-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
   ---------------------------------------- 0.

In [ ]:
from django.db import models
from django.core.validators import RegexValidator

class User(models.Model):
    name = models.CharField(max_length=255)
    phone_regex = RegexValidator(regex=r'^\d{11}$', message="Phone number must be 11 digits.")
    phone_number = models.CharField(validators=[phone_regex], max_length=11, unique=True)
    email = models.EmailField(max_length=255, unique=True)
    password = models.CharField(max_length=255)
    date_of_birth = models.DateField()

    def __str__(self):
        return self.name

In [ ]:
from django import forms
from .models import User
import re

class RegistrationForm(forms.ModelForm):
    class Meta:
        model = User
        fields = ['name', 'phone_number', 'email, 'password', 'date_of_birth']

    # Validate password to contain uppercase, lowercase, digit, and special character
    def clean_password(self):
        password = self.cleaned_data.get('password')
        if not re.match(r'^(?=.*[a-z])(?=.*[A-Z])(?=.*\d)(?=.*[@$!%*?&])[A-Za-z\d@$!%*?&]{8,}$', password):
            raise forms.ValidationError('Password must contain one uppercase, lowercase, digit, and special character.')
        return password

    # Ensure user is at least 18 years old
    def clean_date_of_birth(self):
        dob = self.cleaned_data.get('date_of_birth')
        from datetime import date
        if dob > date.today().replace(year=date.today().year - 18):
            raise forms.ValidationError('You must be at least 18 years old.')
        return dob

    # Ensure the email is valid and from Gmail, Yahoo, or Outlook
    def clean_email(self):
        email = self.cleaned_data.get('email')
        if not re.match(r'.*@gmail\.com$|.*@yahoo\.com$|.*@outlook\.com$', email):
            raise forms.ValidationError('Only Gmail, Yahoo, or Outlook emails are allowed.')
        return email

In [ ]:
from django.shortcuts import render, redirect
from .forms import RegistrationForm
from .models import User
from django.core.mail import send_mail
from django.utils.http import urlsafe_base64_encode, urlsafe_base64_decode
from django.utils.encoding import force_bytes, force_str

def register(request):
    if request.method == 'POST':
        form = RegistrationForm(request.POST)
        if form.is_valid():
            # Hash the password before saving
            user = form.save(commit=False)
            from django.contrib.auth.hashers import make_password
            user.password = make_password(form.cleaned_data.get('password'))
            user.save()

            # Send verification email
            send_verification_email(user)

            return redirect('registration_success')
    else:
        form = RegistrationForm()
    return render(request, 'users/register.html', {'form': form})

In [ ]:
def send_verification_email(user):
    verification_token = urlsafe_base64_encode(force_bytes(user.id))
    verification_link = f"http://example.com/verify/{verification_token}"
    send_mail(
        'Verify your email',
        f'Click this link to verify your account: {verification_link}',
        'admin@example.com',
        [user.email],
        fail_silently=False,
    )

In [ ]:
def verify(request, token):
    try:
        user_id = force_str(urlsafe_base64_decode(token))
        user = User.objects.get(pk=user_id)
        # Mark user as verified (you would need to add a field in the model for this)
        user.is_verified = True
        user.save()
        return redirect('verification_success')
    except (TypeError, ValueError, OverflowError, User.DoesNotExist):
        return render(request, 'users/verification_failed.html')
